### Define imports

In [ ]:
%useLatestDescriptors
%use dataframe
%trackExecution generated
%use lib-ext(0.11.0-398)


### Explore the data

In [ ]:
import org.jetbrains.kotlinx.dataframe.codeGen.generateCode
import org.jetbrains.kotlinx.dataframe.codeGen.generateInterfaces

val df = DataFrame.read("/Users/urs/development/github/ai/kotlin-ai-talk/langchain4j/src/main/resources/dataset/burglaries.csv", delimiter = ';')
df.head()

In [ ]:
df.describe()

In [ ]:
df.select("ArrestMade", "Datetime", "AmountStolen", "CategoryOfItemsStolen", "TypeOfProperty")

In [ ]:
df.generateCode()


In [ ]:
//
df.select { ArrestMade and Datetime and AmountStolen and CategoryOfItemsStolen and TypeOfProperty}


In [ ]:
import kotlinx.datetime.DayOfWeek


val dfExtra = df.add("DayOfWeek"){Datetime.dayOfWeek}.add("hourOfDay"){Datetime.hour}.add("Date"){Datetime.date}
dfExtra.generateCode()
dfExtra.generateInterfaces()


In [ ]:
dfExtra.select{hourOfDay and DayOfWeek and AmountStolen and CategoryOfItemsStolen and TypeOfProperty}


### Visualize Data

In [ ]:
%use kandy

In [ ]:
dfExtra.plot {
    points {
        x(Datetime){
            axis.name = "Day"
        }
        y(CategoryOfItemsStolen) {
            axis.name = "Category of Stolen Goods"
        }
        size(AmountStolen){
            scale = continuous(range = 4.0..50.0)
        }
        color(ArrestMade) {
            scale = continuous(range = Color.BLUE..Color.RED)
        }
        symbol(TypeOfProperty) {
            legend {
                type = LegendType.DiscreteLegend()
                name = "Type Of Property"
            }
        }
    }
    layout.size = 1200 to 900
}

### Figure out *WHEN*

In [ ]:
val narrowedDf = dfExtra
    .filter {
        CategoryOfItemsStolen == "food"
                && ArrestMade == false
                && Date > LocalDate.parse("2024-08-13")}

In [ ]:
import dev.example.utils.getDayPeriodAll

val result = narrowedDf.add("periodOfDay"){getDayPeriodAll(hourOfDay)}
    .groupBy("DayOfWeek", "periodOfDay")
    .aggregate { count() into "count" }

// Print the result
result.print()

In [ ]:
result.plot {
    x(DayOfWeek) {
        axis {
            name = "Date of Week" // Axis label
        }
    }
    y(count) {
        axis.name = "Food Crimes Count"
    }

    bars {
        fillColor("periodOfDay") {
            legend {
                type = LegendType.DiscreteLegend()
                name = "Period of Day"
            }
        }
        //position = Position.stack()
    }
    layout.size = 1200 to 700
}

### Figure out *WHERE*

In [ ]:
import java.net.URI
import javax.imageio.ImageIO
import dev.example.utils.*

val markers = narrowedDf.map {googleMapsMarker(
        label = TypeOfProperty, 
        lat = Latitude, 
        lon = Longitude) 
    }.joinToString("")

val url = "https://maps.google.com/maps/api/staticmap?center=Finance+District,New+York,NY&zoom=14&size=512x512&maptype=roadmap$markers&sensor=false&key=$googleMapsKey"
Image(ImageIO.read(URL(url).openStream())).withWidth(600)